In [1]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.models import load_model

2023-11-18 00:16:19.860682: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-18 00:16:20.331779: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-18 00:16:20.331848: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-18 00:16:20.334523: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-18 00:16:20.586125: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-18 00:16:22.874998: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

In [6]:
# Define your folder structure
data_dir = 'data'
classes = [
    "Andalusi",
    "Rai",
    "Chaabi",
    "Gnawa",
    "Aita",
    "Moroccan Pop",
    "Moroccan Rap",
    "Moroccan Rock",
    "Amazigh Folk"
]

import tqdm

# Load and preprocess audio data
def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []
    
    for i, class_name in tqdm.tqdm(enumerate(classes)):
        class_dir = os.path.join(data_dir, class_name)
        for filename in tqdm.tqdm(os.listdir(class_dir)):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)
                # Perform preprocessing (e.g., convert to Mel spectrogram and resize)
                mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
                mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
                data.append(mel_spectrogram)
                labels.append(i)
    
    return np.array(data), np.array(labels)

In [7]:
# Split data into training and testing sets
data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))  # Convert labels to one-hot encoding
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

100%|██████████| 4/4 [00:04<00:00,  1.06s/it]
9it [00:42,  4.72s/it]


In [9]:
# Create a neural network model
input_shape = X_train[0].shape
input_layer = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(input_layer, output_layer)

In [10]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

In [11]:
# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Epoch 1/200


2023-11-18 00:21:46.491839: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 60963840 exceeds 10% of free system memory.
2023-11-18 00:21:47.197926: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30481920 exceeds 10% of free system memory.
2023-11-18 00:21:47.198119: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 60963840 exceeds 10% of free system memory.


1/1 [==============================] - 4s 4s/step - loss: 18.0210 - accuracy: 0.0667 - auc: 0.5281 - precision: 0.0690 - recall: 0.0667 - val_loss: 262.4921 - val_accuracy: 0.0000e+00 - val_auc: 0.4375 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/200


2023-11-18 00:21:48.236198: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 60963840 exceeds 10% of free system memory.
2023-11-18 00:21:48.859794: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30481920 exceeds 10% of free system memory.


1/1 [==============================] - 1s 1s/step - loss: 76.3762 - accuracy: 0.2333 - auc: 0.5831 - precision: 0.2333 - recall: 0.2333 - val_loss: 238.0031 - val_accuracy: 0.2500 - val_auc: 0.5742 - val_precision: 0.2500 - val_recall: 0.2500
Epoch 3/200
1/1 [==============================] - 1s 926ms/step - loss: 71.6408 - accuracy: 0.3333 - auc: 0.6176 - precision: 0.3333 - recall: 0.3333 - val_loss: 157.3022 - val_accuracy: 0.3750 - val_auc: 0.6094 - val_precision: 0.4286 - val_recall: 0.3750
Epoch 4/200
1/1 [==============================] - 1s 881ms/step - loss: 25.5342 - accuracy: 0.5333 - auc: 0.7497 - precision: 0.5333 - recall: 0.5333 - val_loss: 142.1520 - val_accuracy: 0.2500 - val_auc: 0.5996 - val_precision: 0.2500 - val_recall: 0.2500
Epoch 5/200
1/1 [==============================] - 1s 945ms/step - loss: 15.3580 - accuracy: 0.7333 - auc: 0.8680 - precision: 0.7333 - recall: 0.7333 - val_loss: 136.1222 - val_accuracy: 0.2500 - val_auc: 0.5645 - val_precision: 0.2857 - va

In [14]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.125


In [ ]:
# Save the model
model.save('audio_classification_model.h5')

In [ ]:
# Load the saved model
model = load_model('audio_classification_model.h5')

# Define the target shape for input spectrograms
target_shape = (128, 128)

# Define your class labels
classes = ['cat', 'dog']

# Function to preprocess and classify an audio file
def test_audio(file_path, model):
    # Load and preprocess the audio file
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
    mel_spectrogram = tf.reshape(mel_spectrogram, (1,) + target_shape + (1,))
    
    # Make predictions
    predictions = model.predict(mel_spectrogram)
    
    # Get the class probabilities
    class_probabilities = predictions[0]
    
    # Get the predicted class index
    predicted_class_index = np.argmax(class_probabilities)
    
    return class_probabilities, predicted_class_index

# Test an audio file
test_audio_file = 'dog_barking_4.wav'
class_probabilities, predicted_class_index = test_audio(test_audio_file, model)

# Display results for all classes
for i, class_label in enumerate(classes):
    probability = class_probabilities[i]
    print(f'Class: {class_label}, Probability: {probability:.4f}')

# Calculate and display the predicted class and accuracy
predicted_class = classes[predicted_class_index]
accuracy = class_probabilities[predicted_class_index]
print(f'The audio is classified as: {predicted_class}')
print(f'Accuracy: {accuracy:.4f}')